# This code for blur $^{40}Ar$ Monte Carlo parameters
# Код для размытия Монте Карло данных $^{40}Ar$

### Import required libraries
### Подключаем необходимые библиотеки

In [1]:
import random
import uproot
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pylab as plt
from scipy.stats import chisquare

import math
from scipy.special import gamma

from scipy.optimize import curve_fit

import numpy.random as ra

import sys
sys.setrecursionlimit(10000)

from itertools import chain

import scipy.stats as st

import cellbell

### Import datasets 
### Импортируем наборы данных

In [2]:
ds_Ar40_m100 = pd.read_csv('/home/kingaa/Desktop/DarkSide/Monte/data/outputWIMP_400kWIMP_m100.csv',header = 0, sep = ",")
ds_Ar40_m200 = pd.read_csv('/home/kingaa/Desktop/DarkSide/Monte/data/outputWIMP_400kWIMP_m200.csv',header = 0, sep = ",")
ds_Ar40_m500 = pd.read_csv('/home/kingaa/Desktop/DarkSide/Monte/data/outputWIMP_400kWIMP_m500.csv',header = 0, sep = ",")
ds_Ar40_m1000 = pd.read_csv('/home/kingaa/Desktop/DarkSide/Monte/data/outputWIMP_400kWIMP_m1000.csv',header = 0, sep = ",")

ds_Ar40_m100 = ds_Ar40_m100.dropna()
ds_Ar40_m200 = ds_Ar40_m200.dropna()
ds_Ar40_m500 = ds_Ar40_m500.dropna()
ds_Ar40_m1000 = ds_Ar40_m1000.dropna()

ds_Ar40_mall = pd.concat([ds_Ar40_m100, ds_Ar40_m200, ds_Ar40_m500, ds_Ar40_m1000])

ds_Ar40 = ds_Ar40_mall
ds_Ar40 = ds_Ar40.query('s1>30 and s1 < 500 and f90 > 0.5 and f90 < 0.82')

Ar40_f90 = list(ds_Ar40['f90'])
Ar40_f30 = list(ds_Ar40['f30'])
Ar40_f60 = list(ds_Ar40['f60'])
Ar40_f200 = list(ds_Ar40['f200'])
Ar40_s1 = list(ds_Ar40['s1'])
Ar40_s2 = list(ds_Ar40['s2'])
Ar40_late = list(ds_Ar40['s1late'])
Ar40_prompt = list(ds_Ar40['s1prompt'])

ds_AmBe = "/home/kingaa/Desktop/DarkSide/Monte/AmBe_cut_500.root"
tree_AmBe = uproot.open(ds_AmBe)["TreeB"]

df_f90_ds_AmBe = tree_AmBe.pandas.df('f90')
df_f90_ds_AmBe['f30'] = tree_AmBe.pandas.df('f30')
df_f90_ds_AmBe['f60'] = tree_AmBe.pandas.df('f60')
df_f90_ds_AmBe['f200'] = tree_AmBe.pandas.df('f200')
df_f90_ds_AmBe['s1'] = tree_AmBe.pandas.df('s1')
df_f90_ds_AmBe['s2'] = tree_AmBe.pandas.df('s2')
df_f90_ds_AmBe['sprompt'] = tree_AmBe.pandas.df('long_prompt')
df_f90_ds_AmBe['slate'] = tree_AmBe.pandas.df('long_late')
df_f90_ds_AmBe = df_f90_ds_AmBe.query('s1>30 and s1 < 500 and f90 > 0.5 and f90 < 0.82')

AmBe_f90 = list(df_f90_ds_AmBe['f90'])
AmBe_f30 = list(df_f90_ds_AmBe['f30'])
AmBe_f60 = list(df_f90_ds_AmBe['f60'])
AmBe_f200 = list(df_f90_ds_AmBe['f200'])
AmBe_s1 = list(df_f90_ds_AmBe['s1'])
AmBe_s2 = list(df_f90_ds_AmBe['s2'])
AmBe_sprompt = list(df_f90_ds_AmBe['sprompt'])
AmBe_slate = list(df_f90_ds_AmBe['slate'])

### Fit functions creation
### Создаем фитирующие функции

#### DEAP function
#### Функция распределения коллаборации DEAP
##### DEAP collaboration "Search for dark matter with a 231-day exposure of liquid argon using DEAP-3600 at SNOLAB", 2019.

In [5]:
def deap_distr_NR(f, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, A):
    global q 
    fbar = a0 + a1/(q - a2) + a3/(q - a4)**2
    fbarnew = 1 - fbar
    b = a5 + a6/q + a7/q**2
    sigma = a8 + a9/q + a10/q**2
    fnr = A * 1/math.sqrt(2*math.pi*sigma**2) * np.exp(-(f**2)/(2*sigma**2)) * (fbar**b * (1 - f)**(b-1) * np.exp(-fbar*(1 - f)))/(gamma(b))
    #fnr = 1/np.sqrt(2*math.pi*sigma**2) * np.exp(-(f**2)/(2*sigma**2)) 
    return fnr

#### David Hinkley function 
#### Функция распределения Дэвида Хинкли
##### W. H. Lippincott et. al. "Scintillation time dependence and pulse shape discrimination in liquid argon", 2008.

In [6]:
def hinkley(x, mu_s1late_NR, mu_s1prompt_NR, sigma_s1late_NR,sigma_s1prompt_NR, A):
    func1 = A * np.exp((-0.5*(mu_s1late_NR*x-mu_s1prompt_NR*(1-x))**2)/(sigma_s1late_NR**2*x**2+sigma_s1prompt_NR**2*(1-x)**2))*(sigma_s1late_NR**2*mu_s1prompt_NR*x+sigma_s1prompt_NR**2*mu_s1late_NR*(1-x))/((2*np.pi)**(0.5)*(sigma_s1late_NR**2*x**2+sigma_s1prompt_NR**2*(1-x)**2)**1.5)
    return func1

### Main code for $f_{30}$, $f_{60}$, $f_{90}$, $f_{200}$
### Основной код для $f_{30}$, $f_{60}$, $f_{90}$, $f_{200}$

In [ ]:
# Inverse transform sampling for hinkley distribution
# Создаем функцию выборки обратного преобразования
def its_hinkley_NR(N):
    prob = f/float(sum(f))
    cum_prob = np.cumsum(prob)   
    R = ra.uniform(0, 1, N)
    gen_random = [float(x[np.argwhere(cum_prob == min(cum_prob[(cum_prob - r) > 0]))]) for r in R] 
    return gen_random
# Blur function for 'extra values' - values that gets into a crowded bin
# Создаем функцию размытия для 'экстра значений' - значений, которые попали в переполненный бин
def blur(value, depth):
    j = int(np.argwhere(list_AmBe_data[:100] == max(list_AmBe_data[:100][(list_AmBe_data[:100] - value) <= 0])))
    if (n_list[j] < n_AmBe_data[j]) or depth == 10:
        return j, value
    else:
        depth = depth + 1
        a = its_hinkley_NR(1)[0]
        return blur(a, depth)
# Function for comparison plotting data before and after blur
# Функция для сравнительного построения данных до и после размытия
def plotting (AmBe_data, Ar40_old, Ar40_new):
    nbins = 50
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    n_data_aar, list_data_aar, patches_data_aar = plt.hist(AmBe_data, bins = nbins, density = True, histtype = 'step', color = 'black', linewidth = 3, label = 'AAr')
    n_data_ar40, list_data_ar40, patches_data_ar40 = plt.hist(Ar40_old, bins = nbins, density = True, histtype = 'step', color = 'red', linewidth = 3, label = 'Ar39')
    plt.title('$^{39}$Ar $f_{30}$ до преобразований', fontsize=22)
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.legend()
    plt.subplot(122)
    n_data_aar, list_data_aar, patches_data_aar = plt.hist(AmBe_data, bins = nbins, density = True, histtype = 'step', color = 'black', linewidth = 3, label = 'AAr')
    n_new_data_ar40, list_new_data_ar40, patches_new_data_ar40 = plt.hist(Ar40_new, bins = nbins, density = True, histtype = 'step', color = 'red', linewidth = 3, label = 'Ar39')
    plt.title('$^{39}$Ar $f_{30}$ после преобразований', fontsize=22)
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.legend()

In [ ]:
parameter_AmBe_list = ['AmBe_f30', 'AmBe_f60', 'AmBe_f90', 'AmBe_f200', 'AmBe_s1']
Ar40_new_parameters = []
global n_list
for parameter in parameter_AmBe_list: 
    global data
    n_AmBe_data, list_AmBe_data, patches_AmBe_data = plt.hist(parameter, bins = 100, density = False)
    popt_AmBe, pcov_AmBe = curve_fit(hinkley, list_AmBe_data[:100], n_AmBe_data, maxfev=100000)
    n_AmBe_data_fit = hinkley(list_AmBe_data, *popt_AmBe)
    global x, f
    x = np.linspace(min(parameter), max(parameter), 200)
    f = hinkley(x, *popt_AmBe)
    if (parameter == 'AmBe_f30'):
        data = Ar40_f30
    elif (parameter == 'AmBe_f60'):
        data = Ar40_f60
    elif (parameter == 'AmBe_f90'):
        data = Ar40_f90
    elif (parameter == 'AmBe_f200'):
        data = Ar40_f200
    data1 = data[:len(AmBe_f90)]
    data2 = data[len(AmBe_f90):2*len(AmBe_f90)]
    data3 = data[2*len(AmBe_f90):3*len(AmBe_f90)]
    data_lists = [data1, data2, data3]
    for data in data_lists:
        data_new = []
        n_list = [0] * 101
        for i in range(len(data)):
            j, value = blur(data[i], 0)
            n_list[j] = n_list[j] + 1
            data_new.append(value)
        new_param = list(chain(*data_new))
    Ar40_new_parameters.append(new_param)

### Main code for $S_1$
### Основной код для $S_1$

In [ ]:
def deap_random_hinkley_NR_s1(N):
    prob = f_hinkley_NR/float(sum(f_hinkley_NR))
    cum_prob = np.cumsum(prob)    
    R = ra.uniform(0, 1, N)
    gen_random = [float(x_hinkley_NR[np.argwhere(cum_prob == min(cum_prob[(cum_prob - r) > 0]))]) for r in R]  
    return gen_random
def blur_s1(value, depth):
    if value == min(list_s1_AmBe):
        value = value + 1
    if value == max(list_s1_AmBe):
        value = value - 1
    j = int(np.argwhere(list_s1_AmBe[:200] == max(list_s1_AmBe[:200][(list_s1_AmBe[:200] - value) <= 0])))
    if (n_list[j] < n_s1_AmBe_trial_deap[j]) or depth == 5:
        return j, value
    else:
        depth = depth + 1
        a = deap_random_hinkley_NR_s1(1)[0]
        return razmitie_s1(a, depth)

In [ ]:
# Define 'n_AAr_data' - number of events in each bin in AmBe data
# Ищем чему равно n в каждом бине в реальных данных NR
global n_s1_AmBe, list_s1_AmBe, patches_s1_AmBe
n_s1_AmBe, list_s1_AmBe, patches_s1_AmBe = plt.hist(AmBe_s1, 200)
# Fit
# фитируем
popt_s1_hinkley_AmBe, pcov_s1_hinkley_AmBe = curve_fit(hinkley, list_s1_AmBe[:200], n_s1_AmBe, maxfev=1000000)
# Calculate n for fit function in each bin
# вычисляем n для фитаglobal x_hinkley_NR,f_hinkley_NR
x_hinkley_NR = np.linspace(min(Ar40_s1), max(Ar40_s1), 200)
f_hinkley_NR = hinkley(x_hinkley_NR, *popt_s1_hinkley_AmBe)
# Divide Ar40 dataset on 3 different datasets
# Разделяем набор данных Ar40 на 3 различных набора данных
data1_s1 = Ar40_s1[:len(AmBe_f90)]
data2_s1 = Ar40_s1[len(AmBe_f90):2*len(AmBe_f90)]
data3_s1 = Ar40_s1[2*len(AmBe_f90):3*len(AmBe_f90)]
data_s1_list = [data1_s1, data2_s1, data3_s1]
global n_list
a = []
for data in data_s1_list:
    data_new = []
    n_list = [0] * 201
    for i in range (len(data)):
        j, value = blur_s1(data[i], 0)
        n_list[j] = n_list[j] + 1
        data_new.append(value)
    a.append(data_new)
Ar40_new_s1 = list(chain(*a))

In [ ]:
# Create the pandas DataFrame and save it
# Создание нового датафрейма и сохранение его в .csv файл
data = {'f30':Ar40_new_parameters[0], 'f60':Ar40_new_parameters[1], 'f90':Ar40_new_parameters[2], 'f200':Ar40_new_parameters[3], 's1':Ar40_new_s1}
df = pd.DataFrame(data) 
df.to_csv('new_ds_Ar40_MC_30_500_f30_f60_f90_f200_s1.csv')